In [ ]:
%%time

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# takes around 45 seconds
model = VGG16(weights='imagenet', include_top=False)

model.summary()

In [ ]:
%%time
import numpy as np

IMG_SIZE = 50
TRAIN_DATA_PATH = r'D:\LICENTA\processed_data\size_{size1}x{size2}\train_data.npy'.format(size1=IMG_SIZE, size2=IMG_SIZE)
# this takes around 1 minute
images = np.load(TRAIN_DATA_PATH)

In [ ]:
import pandas as pd

train_biz = pd.read_csv('train.csv')
train_biz.dropna(axis=0, inplace=True)
business_ids = train_biz['business_id'].values

In [ ]:
%%time
from tqdm import tqdm
# around 2,5 hours on the current dataset
biz_to_features = []
feature_imgs_list = []

for business_id in tqdm(list(business_ids)):
    matched_images = images[images[:,1] == business_id][:,0]
    
    for img in list(matched_images):
        img = img.astype(dtype='float32')
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        img_features = model.predict(img)
        feature_imgs_list.append(img_features)
    
    biz_to_features.append([np.array(business_id), np.mean(np.array(feature_imgs_list), axis=0)])

In [ ]:
FEATURES_PATH = r'D:\LICENTA\processed_data\size_{size1}x{size2}\biz_to_features.npy'.format(size1=IMG_SIZE, size2=IMG_SIZE)

np.save(FEATURES_PATH, np.array(biz_to_features, dtype='object'))

In [ ]:
data = np.load(FEATURES_PATH)

data[0]